In [ ]:
import sys

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from gensim.models.fasttext import FastText

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Indonesian FastText Downloading and Loading

## Loads FastText Model

In [6]:
model = FastText.load_fasttext_format('/content/drive/My Drive/TA/FastTextIndonesia/cc.id.300.bin')
# model = FastText.load_fasttext_format('/content/drive/My Drive/TA/FastTextIndonesia/wiki.id.bin')
# model = FastText.load_fasttext_format('/content/drive/My Drive/TA/FastTextIndonesia/cc.id.100.bin')
# model = FastText.load_fasttext_format('/content/drive/My Drive/TA/FastTextIndonesia/wiki.id.100.bin')

EMBEDDING_SIZE = len(model.wv['a'])
print(EMBEDDING_SIZE)

300


In [7]:
print("algoritmatik" in model.wv.vocab)
# print(model.wv.most_similar("algoritmatik"))

False


# Loading Data

In [8]:
SQUAD_DATASET_PATH = '/content/drive/My Drive/TA/train-v2.0-translated_fixed.json'

df_squad = pd.read_json(SQUAD_DATASET_PATH)
# df_squad = df_squad.drop(columns=['failure_percentage', 'total_questions'])
print(df_squad.shape)
print(df_squad)

(442, 2)
                                           title                                         paragraphs
0                                        Beyonce  [{'qas': [{'question': 'Kapan Beyonce mulai me...
1                                Frédéric Chopin  [{'qas': [{'question': 'Bagaimana kewarganegar...
2    Hubungan Tiongkok-Tibet selama dinasti Ming  [{'qas': [{'question': 'Siapakah Wang Jiawei d...
3                                           IPod  [{'qas': [{'question': 'Perusahaan mana yang m...
4         The Legend of Zelda: Twilight Princess  [{'qas': [{'question': 'Apa kategori game Lege...
..                                           ...                                                ...
437                                      Infeksi  [{'qas': [{'question': 'Dari sejumlah besar mi...
438                                      Berburu  [{'qas': [{'question': 'Apa praktik membunuh a...
439                                    Kathmandu  [{'qas': [{'question': 'Negara mana Kathm

# Delete Unfound Answers from Dataset

In [10]:
total_questions = 0
for taken_topic_idx in range(df_squad.shape[0]):
    for taken_context_idx in range(len(df_squad.iloc[taken_topic_idx]['paragraphs'])):
        i = 0
        qas = df_squad.iloc[taken_topic_idx]['paragraphs'][taken_context_idx]['qas']
        while i < len(qas):
            indonesian_answer = qas[i].get('indonesian_answers') or qas[i].get('indonesian_plausible_answers')
            if indonesian_answer[0]['answer_start'] < 0:
                qas.pop(i)
            else:
                i += 1
                total_questions += 1
print(total_questions)

120818


# Prepare Tensors and Delete Unfound Answers Once Again

In [ ]:
print(model.wv.most_similar('<unk'))
print(model.wv.most_similar('<sos>'))
print(model.wv.most_similar('<eos>'))
print(model.wv.most_similar('</sos>'))
print(model.wv.most_similar('</eos>'))

In [ ]:
SOS_TOKEN = 0
EOS_TOKEN = 1


class TextDict:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addWords(self, list_of_words):
        for word in list_of_words:
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
import re
import unicodedata

# Complete punctuation from string.punctiation: !"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

get_non_ascii_regex = re.compile(r"[^\x00-\x7F\u2013]")
punctuations = '!"#$%&\()*+,./:;<=>?@[\\]^_`{|}~'
''' tokenize(s)
    1. Convert convertable non-ASCII unicode to ASCII,
    2. Separate some punctuations (with some cases) with space,
    3. Delete non-ASCII characters
'''
def tokenize(s):
    s = unicodeToAscii(s)
    s = re.sub(r"([%s])" % punctuations, r" \1 ", s)
    s = re.sub(r"([^a-zA-Z0-9]')", r" \1 ", s)
    s = re.sub(get_non_ascii_regex, r"", s)
    s = s.split()
    return s
tokenize('"Frédéric Chopin! adalah, (1907–1986) (1907-1986) anak-anak (children): gembala. Andi\'s sheep is an \'03 R&B player; 奉獻 km² Jīdū °5 вера ʰp"')

In [14]:
context_length = []
for paragraph in df_squad['paragraphs']:
    for qas in paragraph:
        context_length.append(len(tokenize(qas['context'])))
df_context_length = pd.DataFrame(context_length)
context_length_desc = df_context_length.describe()
print(context_length_desc, end='\n\n')

CONTEXT_MAX_LENGTH = df_context_length.max()[0]
print(CONTEXT_MAX_LENGTH)

question_length = []
for paragraph in df_squad['paragraphs']:
    for qas in paragraph:
        for qa in qas['qas']:
            question_length.append(len(tokenize(qa['question'])))
df_question_length = pd.DataFrame(question_length)
question_length_desc = df_question_length.describe()
print(question_length_desc, end='\n\n')

QUESTION_MAX_LENGTH = df_question_length.max()[0]
print(QUESTION_MAX_LENGTH)

                  0
count  19035.000000
mean     122.194799
std       52.583681
min       19.000000
25%       90.000000
50%      112.000000
75%      147.000000
max      743.000000

743
                   0
count  120818.000000
mean        9.813604
std         3.261368
min         1.000000
25%         8.000000
50%         9.000000
75%        12.000000
max        58.000000

58


In [ ]:
from fuzzywuzzy import fuzz

WORD_SIMILARITY_THRESHOLD = 80

def get_answer_index_from_context(tokenized_context, tokenized_answers, _position):
    position = _position
    i = 0
    j = 0
    while i < len(tokenized_context) and j < min(2, len(tokenized_answers)):
        if position <= 5:
            if tokenized_context[i].isnumeric():
                similarity = fuzz.ratio(tokenized_context[i], tokenized_answers[j])
            else:
                similarity = fuzz.partial_ratio(tokenized_context[i], tokenized_answers[j])
            # print(f'{tokenized_context[i]} vs {tokenized_answers[j]} = {similarity}')
            if similarity >= WORD_SIMILARITY_THRESHOLD:
                j += 1
        position -= len(tokenized_context[i]) + 1
        i += 1
    if j >= min(2, len(tokenized_answers)):
        return i-j
    else:
        return -1

In [ ]:
# for topic in df_squad.iloc[2]['paragraphs']:
#     print(topic['context'])

In [ ]:
import string
import time


input_textdict = TextDict('context')
output_textdict = TextDict('question')

input_tensors = []
answer_tensors = []
target_tensors = []
# case_tensors = []
# is_number_tensors = []

not_found = 0

start_time = time.time()
for taken_topic_idx in range(df_squad.shape[0]):
    for taken_context_idx in range(len(df_squad.iloc[taken_topic_idx]['paragraphs'])):
        context = df_squad.iloc[taken_topic_idx]['paragraphs'][taken_context_idx]['context']
        tokenized_context = tokenize(context)
        input_textdict.addWords(tokenized_context)
        count_tobe_removed_chars = len(re.findall(get_non_ascii_regex, unicodeToAscii(context))) * 1.5  # With assumption every nonascii is followed by space
        context_tensor = [input_textdict.word2index[word] for word in tokenized_context]
        context_tensor.extend([0 for _ in range(CONTEXT_MAX_LENGTH - len(context_tensor))])
        # context_tensor = convert_word_to_word_embedding(tokenized_context, model, context_tensor_target_shape, 'Context', taken_topic_idx, taken_context_idx)

        qas = df_squad.iloc[taken_topic_idx]['paragraphs'][taken_context_idx]['qas']
        i = 0
        while i < len(qas):
            qa = qas[i]

            indonesian_question = qa['question']
            tokenized_questions = tokenize(indonesian_question)
            output_textdict.addWords(tokenized_questions)
            question_tensor = [output_textdict.word2index[word] for word in tokenized_questions]
            question_tensor.extend([0 for _ in range(QUESTION_MAX_LENGTH - len(question_tensor))])
            # question_tensor = convert_word_to_word_embedding(tokenized_questions, model, question_tensor_target_shape, 'Question', taken_topic_idx, taken_context_idx, i)

            indonesian_answer = qa.get('indonesian_answers') or qa.get('indonesian_plausible_answers')
            tokenized_answers = tokenize(indonesian_answer[0]['text'])
            answer_start = indonesian_answer[0]['answer_start'] - count_tobe_removed_chars
            try:
                word_idx = get_answer_index_from_context(tokenized_context, tokenized_answers, answer_start)
            except IndexError as e:
                word_idx = -1
                not_found += 1
                qas.pop(i)
                print(f'Exception {e} at topic_idx={taken_topic_idx}, taken_context_idx={taken_context_idx}, qas_idx={i}')
                # i += 1
                continue
            if word_idx == -1:
                not_found += 1
                qas.pop(i)
                print(f'Not found at topic_idx={taken_topic_idx}, taken_context_idx={taken_context_idx}, qas_idx={i}')
                # i += 1
                continue
            answer_tensor = [1 if i in range(word_idx, word_idx+len(tokenized_answers)) else 0 for i in range(CONTEXT_MAX_LENGTH)]

            input_tensors.append(context_tensor)
            answer_tensors.append(answer_tensor)
            target_tensors.append(question_tensor)

            i += 1
end_time = time.time()

total_questions = total_questions - not_found
print(f'Not found answers: {not_found}. Questions left: {total_questions - not_found}')
print(f'Execution time: {end_time-start_time}')

In [20]:
feature_tensors = np.array(answer_tensors).reshape(len(answer_tensors), -1, 1)

input_tensors = torch.from_numpy(np.array(input_tensors)).long().to(device)
feature_tensors = torch.tensor(feature_tensors).float().to(device)
target_tensors = torch.from_numpy(np.array(target_tensors)).long().to(device)
print(input_tensors.size())
print(feature_tensors.size())
print(target_tensors.size())

# input_tensors = np.array(input_tensors)
# answer_tensors = np.array(answer_tensors)
# target_tensors = np.array(target_tensors)
# print(input_tensors.shape)
# print(answer_tensors.shape)
# print(target_tensors.shape)

# print(len(input_tensors))
# print(len(answer_tensors))
# print(len(target_tensors))

torch.Size([120765, 743])
torch.Size([120765, 743, 1])
torch.Size([120765, 58])


In [21]:
# Count rows which contain non-zero element
np.unique(np.array(answer_tensors).nonzero()[0]).shape

(120746,)

# Create Weights Matrix Embedding from FastText

In [ ]:
OOV_REPLACEMENT = '<unk>'

print(len(input_textdict.index2word))
print(len(output_textdict.index2word))

input_weights_matrix = np.zeros((len(input_textdict.index2word), EMBEDDING_SIZE))
output_weights_matrix = np.zeros((len(output_textdict.index2word), EMBEDDING_SIZE))

def convert_word_to_word_embedding(word, model):
    try:
        word_embedding = model.wv[word]
    except KeyError as e:
        print(f'{type(e).__name__} exception: {e}.')
        word_embedding = model.wv[OOV_REPLACEMENT]
    return word_embedding

for i, word in input_textdict.index2word.items():
    input_weights_matrix[i] = convert_word_to_word_embedding(word, model)
for i, word in output_textdict.index2word.items():
    output_weights_matrix[i] = convert_word_to_word_embedding(word, model)
print(f'All unknown words will be replaced with {OOV_REPLACEMENT}')

input_weights_matrix = torch.from_numpy(input_weights_matrix)
output_weights_matrix = torch.from_numpy(output_weights_matrix)
print(input_weights_matrix.size())
print(output_weights_matrix.size())

# print(input_weights_matrix.shape)
# print(output_weights_matrix.shape)

# Sequence to Sequence

In [ ]:
def create_emb_layer(weights_matrix, non_trainable=False):
    vocab_size, embedding_dim = weights_matrix.size()
    emb_layer = nn.Embedding(vocab_size, embedding_dim)
    emb_layer.load_state_dict({'weight': weights_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, vocab_size, embedding_dim

## Encoder

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, weights_matrix, feature_tensors, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding, _, embedding_dim = create_emb_layer(weights_matrix)
        intermediate_dim = embedding_dim + feature_tensors.size(2)
        self.lstm = nn.LSTM(intermediate_dim, hidden_size)

    def forward(self, input, feature_tensor, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = torch.cat((embedded, feature_tensor.view(1, 1, -1)), 2)
        print(output.size())
        print(hidden.size())
        output, hidden = self.lstm(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

## Decoder

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, weights_matrix, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding, _, embedding_dim = create_emb_layer(weights_matrix)
        self.lstm = nn.LSTM(embedding_dim, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.lstm(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

## Attention

In [ ]:
MAX_LENGTH = CONTEXT_MAX_LENGTH

class AttnDecoderRNN(nn.Module):
    def __init__(self, weights_matrix, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding, _, embedding_dim = create_emb_layer(weights_matrix)
        self.attn = nn.Linear(embedding_dim + hidden_size, self.max_length)
        self.attn_combine = nn.Linear(embedding_dim + hidden_size, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        # embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((inputs[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.lstm(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# Training

## Helper Function

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return f'{m}m {s}s'


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return f'{asMinutes(s)} (- {asMinutes(rs)})'

## Train

In [ ]:
teacher_forcing_ratio = 0.5


def train(input_tensor, feature_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], feature_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_TOKEN]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_TOKEN:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    input_batch = input_tensors.size(0)

    for iter in range(1, n_iters + 1):
        input_tensor = input_tensors[iter % input_batch]
        feature_tensor = feature_tensors[iter % input_batch]
        target_tensor = target_tensors[iter % input_batch]

        loss = train(input_tensor, feature_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

# Plotting Results

In [ ]:
plt.switch_backend('agg')
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

# Evaluation

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        # input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     feature_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_TOKEN]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_TOKEN:
                decoded_words.append('<eos>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

# Train & Evaluate

In [34]:
hidden_size = 256
encoder1 = EncoderRNN(input_weights_matrix, feature_tensors, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(output_weights_matrix, hidden_size, output_textdict.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

torch.Size([1, 1, 301])
torch.Size([1, 1, 256])


RuntimeError: ignored